<a href="https://colab.research.google.com/github/luanakwon/GreatBarrierReef/blob/main/GBR_YOLOBasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download dataset from kaggle api
kaggle  
https://www.kaggle.com/c/tensorflow-great-barrier-reef/data  

kaggle API usage  
https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb

In [ ]:
!pip install kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c tensorflow-great-barrier-reef

In [ ]:
!unzip -q /content/tensorflow-great-barrier-reef.zip 

# Session 1
---
Useful links  
- [YOLOv5 - Train Custom Data github](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data)  


In [5]:
# create hierarchy
import os

for f1 in 'train', 'val', 'test':
  for f2 in 'images', 'labels':
    os.makedirs(f'/content/Datasets/{f1}/{f2}')

In [6]:
# read images from /content/train_images
# change name
# save the image to train | val | test. ratio = 0.64 : 0.16 : 0.2
# create corresponding txt file and save

In [41]:
def annotation2YoloFormat(_annotation, width=1280, height=720):
  out = ''
  if isinstance(_annotation,str):
    a = _annotation
    while True:
      ix = a.find("'x': ")
      iy = a.find("'y': ")
      iw = a.find("'width': ")
      ih = a.find("'height': ")
      
      if ix < 0 or iy < 0 or iw < 0 or ih < 0:
        break

      x = float(a[ix + len("'x': "):a.find(", 'y")])/width
      y = float(a[iy + len("'y': "):a.find(", 'w")])/height
      w = float(a[iw + len("'width': "):a.find(", 'h")])/width
      h = float(a[ih + len("'height': "):a.find('}')])/height
      
      
      out += '0 %.6f %.6f %.6f %.6f\n'%(x+w/2, y+h/2, w, h)
      a = a[a.find('}')+1:]
    
  return out
annotation2YoloFormat(
    "[{'x': 519, 'y': 155, 'width': 76, 'height': 60}, {'x': 500, 'y': 150, 'width': 70, 'height': 80}]")

'0 0.435156 0.256944 0.059375 0.083333\n0 0.417969 0.263889 0.054688 0.111111\n'

In [7]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.0-218-g7539cd7 torch 1.10.0+cu111 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 70.4/225.9 GB disk)


# Under /content/yolov5

In [ ]:
# create yaml file and save to /data (your at /content/yolov5)
# https://drive.google.com/file/d/1YefSjpQ-MX1y1KqgDzLaQhO-8YSeu4DF/view?usp=sharing
!gdown --id 1YefSjpQ-MX1y1KqgDzLaQhO-8YSeu4DF -O ./data/GBR.yaml

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
# --img : image size (size,size)
# --batch : batch size
# --epochs : epochs
# --data : .yaml file name
# --weights : pretrained weights ('' --cfg yolov5s.yaml if from scratch)
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt

In [ ]:
# visualize
from utils.plots import plot_results 
plot_results('path/to/results.csv')  # plot 'results.csv' as 'results.png'